In [3]:
import pandas as pd
import geopandas as gpd
import time
import os
os.environ['USE_PYGEOS'] = '0'
import warnings
from pandas import read_csv
import numpy as np

warnings.filterwarnings("ignore")
os.chdir('/Users/xiaodanxu/Library/CloudStorage/GoogleDrive-arielinseu@gmail.com/My Drive/GEMS/BILD-AQ/data')
# selected_state = 'MS'

In [4]:
microtype_lookup = read_csv('ccst_geoid_key_tranps_geo_with_imputation.csv', sep = ',')
state_list = microtype_lookup['st_code'].unique().tolist()
processed = ['CA', 'NY', 'WA', 'OR', 'TX', 'AZ', 'NV']
state_list = [x for x in state_list if (x not in processed)]
print(state_list)

['NJ', 'IL', 'MA', 'FL', 'LA', 'VA', 'MD', 'DC', 'DE', 'PA', 'OH', 'RI', 'IN', 'MI', 'WI', 'MN', 'WV', 'NH', 'HI', 'GA', 'NM', 'MS', 'OK', 'AR', 'MO', 'KS', 'KY', 'CO', 'UT', 'NE', 'CT', 'WY', 'IA', 'ID', 'SD', 'ME', 'VT', 'MT', 'ND', 'SC', 'NC', 'TN', 'AK', 'AL']


In [5]:
census_tract_shapefile = 'Network/combined/census_tracts_2017.geojson'
polygon_gdf = gpd.read_file(census_tract_shapefile)
polygon_gdf = polygon_gdf.to_crs('EPSG:4326')
# polygon_gdf.plot()
# from pygris.utils import shift_geometry

# polygon_gdf_rescaled = shift_geometry(polygon_gdf)
# polygon_gdf_rescaled.plot()

In [6]:
# perform spatial intersection
os.chdir('/Volumes/LaCie/project_backup/GEMS_ROUTE/route')
for selected_state in state_list:
    if selected_state == 'OH':
        continue
    print('processing state ' + selected_state)
    output_dir = 'internal/' + selected_state
    route_list = [file for file in os.listdir(output_dir) if file.endswith('.geojson')]
#     print(route_list[0:5])
    start_time = time.time()
    for route in route_list:
        if route.startswith('._'):
            continue
        file_name = route.split('.geojson')[0]
        isExist = os.path.exists(output_dir + '/' + file_name + '.csv')
        if isExist:
            continue
        print('processing route ' + route)
        line_gdf = gpd.read_file(output_dir + '/' + route)
    #     sample_line_gdf = line_gdf.head(100)
        # print(start_time)
        line_by_polygon = gpd.overlay(line_gdf, polygon_gdf, how='intersection')

        # compute segment length in meters
        line_by_polygon = line_by_polygon.to_crs("EPSG:3310") 
        # in order to get length in meter, the shapefile need to re-projected to a coordinate system in meters (not required in R)
        line_by_polygon.loc[:, 'Length'] = line_by_polygon.loc[:, 'geometry'].length
        
        line_by_polygon_df = pd.DataFrame(line_by_polygon.drop(columns='geometry'))
        line_by_polygon_df = line_by_polygon_df[['source',	'destination',
                                                  'distance', 'GEOID', 'Length']]
    #     line_by_polygon_df.columns = []
    #     print(len(line_by_polygon))
        line_by_polygon_df.to_csv(output_dir + '/' + file_name + '.csv', index = False)

    #     break
    end_time = time.time()
    total_time = end_time - start_time
    print('spatial intersection in Python takes ' + str(total_time) + ' sec')

processing state NJ
spatial intersection in Python takes 0.00017499923706054688 sec
processing state IL
spatial intersection in Python takes 0.00013327598571777344 sec
processing state MA
spatial intersection in Python takes 9.703636169433594e-05 sec
processing state FL
spatial intersection in Python takes 0.0016739368438720703 sec
processing state LA
spatial intersection in Python takes 0.00011491775512695312 sec
processing state VA
spatial intersection in Python takes 0.0001227855682373047 sec
processing state MD
spatial intersection in Python takes 0.00013709068298339844 sec
processing state DC
spatial intersection in Python takes 7.295608520507812e-05 sec
processing state DE
spatial intersection in Python takes 8.487701416015625e-05 sec
processing state PA
spatial intersection in Python takes 0.0003509521484375 sec
processing state RI
spatial intersection in Python takes 4.696846008300781e-05 sec
processing state IN
processing route result-9-csv-3_0.geojson
processing route result-

In [5]:
line_by_polygon_df.head(5)

,source,destination,distance,GEOID,Length
0,40143007407,40105172200,99326.245394,40143005900,10072.073880
1,40143007407,40105172300,83091.556274,40143005900,10072.073880
2,40143007407,40105172400,83826.376872,40143005900,10072.073880
3,40143007407,40113940002,109694.228375,40143005900,1325.930235
4,40143007407,40113940004,103867.863385,40143005900,1325.930235
